In [1]:
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [2]:
# from transformers import BlenderbotTokenizer, BlenderbotForConditionalGeneration
# mname = 'facebook/blenderbot-3B'
# model = BlenderbotForConditionalGeneration.from_pretrained(mname)
# tokenizer = BlenderbotTokenizer.from_pretrained(mname)
# UTTERANCE = "My friends are cool but they eat too many carbs."
# inputs = tokenizer([UTTERANCE], return_tensors='pt')
# reply_ids = model.generate(**inputs)
# print(tokenizer.batch_decode(reply_ids))

In [3]:
# inputs

In [4]:
# reply_ids = model.generate(inputs['input_ids'])
# print(tokenizer.batch_decode(reply_ids))

In [5]:
# UTTERANCE = "My friends are cool but they eat too many carbs."
# inputs = tokenizer([UTTERANCE], return_tensors='pt')

In [6]:
# inputs['input_ids'].shape

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import BlenderbotTokenizer, BlenderbotForConditionalGeneration, BlenderbotConfig
import torch
import re
from tqdm import tqdm

import json
import os

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
# blenderbot-400M-distill, "blenderbot-3B", "blenderbot_small-90M"
variant = "blenderbot-3B"
# variant = "blenderbot_small-90M"
out_dir = f"outputs/{variant}"
if not os.path.isdir(out_dir):
    os.makedirs(out_dir)

In [10]:
# tokenizer = AutoTokenizer.from_pretrained(f"facebook/{variant}")
# model = AutoModelForSeq2SeqLM.from_pretrained(f"facebook/{variant}")

model = BlenderbotForConditionalGeneration.from_pretrained(f"facebook/{variant}")
tokenizer = BlenderbotTokenizer.from_pretrained(f"facebook/{variant}")

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [11]:
model=model.to(device)

In [ ]:
stack = []
with open('daily_dialog/test/dialogues_test.txt') as f, open(f'{out_dir}/dialgpt_responses.txt', 'w') as of:
    lines = f.readlines()
    
    for i, line in tqdm(enumerate(lines), total=len(lines)):
        dialog = re.split(" *__eou__ *", line.strip())
        
        predictions = {
            "filename": i,
            "utterances": []
        }
        context = []
        
        dialog = dialog[:-1] # last one is empty
        for step, u in enumerate(dialog):
            if step == 0:
                predictions['utterances'].append(
                    {
                        "true": u
                    }
                )
            else:
                predictions['utterances'][-1]['true'] = u
                
            # No further utterances
            if step == len(dialog) - 1:
                break
                
            # print("\nuser:", u)
            context.append(u)
            # encode the new user input, add the eos_token and return a tensor in Pytorch
            # new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt').to(device)
            new_user_input_ids = tokenizer([u], return_tensors='pt')['input_ids'].to(device)
            
            
            
            # append the new user input tokens to the chat history
            # bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids
            bot_input_ids = torch.cat([bot_input_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids
            
            # generated a response while limiting the total chat history to 1000 tokens, 
            chat_history_ids = model.generate(bot_input_ids[:, -128:]) #blender-3B can't handle more than 128 length

            # pretty print last ouput tokens from bot
            resp = tokenizer.batch_decode(chat_history_ids, skip_special_tokens=True)
            
            for k, s in enumerate(context):
                of.write(f"Turn {k}: {s}\n")
            of.write(f"Blender: {resp}\n\n")
            
            predictions['utterances'].append(
                    {
                        "predicted": resp
                    }
                )
        stack.append(predictions)

In [ ]:
with open(f"{out_dir}/test_predictions.json", "w") as f:
    json.dump(stack, f)